[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay20.ipynb)

# --- Day 20: Jurassic Jigsaw ---

In [1]:
using System.IO;
using System.Numerics;
using System.Text.RegularExpressions;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
class Tile
{
    private char[][] pixels;
    
    public int Id { get; init; }
    public char[][] Pixels { get => pixels; }
    
    public string BorderTop
    {
        get => new string(pixels[0]);
    }
    
    public string BorderBottom
    {
        get => new string(pixels[^1]);
    }
    
    public string BorderLeft
    {
        get => new string(pixels.Select(line => line[0]).ToArray());
    }
    
    public string BorderRight
    {
        get => new string(pixels.Select(line => line[^1]).ToArray());
    }
    
    public string[] Borders
    {
        get => new string[]
        {
            BorderTop,
            BorderRight,
            BorderBottom,
            BorderLeft,
            new string(BorderTop.Reverse().ToArray()),
            new string(BorderRight.Reverse().ToArray()),
            new string(BorderBottom.Reverse().ToArray()),
            new string(BorderLeft.Reverse().ToArray()),
        };
    }
    
    public Tile(string input)
    {
        var lines = input.Split("\n");
        
        // First line as the ID
        Id = Convert.ToInt32(Regex.Match(lines[0], @"\d+").Value);
        
        // Rest of the lines are the pixels
        pixels = lines[1..].Select(line => line.ToCharArray()).ToArray();
    }
    
    // Rotates the tile clockwise one time
    public void Rotate()
    {
        var cols = pixels[0].Length;
        pixels = Enumerable.Range(0, cols).Select(col => Pixels
            .Select(line => line[col])
            .Reverse()
            .ToArray()
        ).ToArray();
    }
    
    // Flips the tile. 0 is the x-axis. 1 is the y-axis.
    public void Flip(int axis)
    {
        switch (axis)
        {
            case 0:
                pixels = pixels.Select(line => line.Reverse().ToArray()).ToArray();
                break;
            case 1:
                pixels = pixels.Reverse().ToArray();
                break;
            default:
                throw new ArgumentException($"Invalid axis: {axis}.");
        }
    }
    
    public override string ToString()
    {
        var pixels = string.Join("\n", Pixels.Select(line => new string(line)));
        return $"Tile {Id}:\n{pixels}";
    }
}

In [3]:
var tiles = File.ReadAllText(@"input/20.txt")
    .Split("\n\n")
    .Select(tile => new Tile(tile))
    .ToArray();

In [4]:
class Image 
{
    public int GridSize { get; init; }
    public Tile[,] Grid { get; init; }
    
    public Dictionary<int, Tile> Tiles { get; init; }
    public Dictionary<string, List<Tile>> TileBorders { get; init; }
    public IEnumerable<Tile> Corners { get; init; }
    
    public Image(IEnumerable<Tile> tiles)
    {
        Tiles = tiles.ToDictionary(t => t.Id, t => t);
        
        GridSize = (int)Math.Sqrt(Tiles.Count());
        Grid = new Tile[GridSize, GridSize];
        
        TileBorders = tiles
            .Select(t => t.Borders.Select(b => (Tile: t, Border: b)))
            .SelectMany(t => t)
            .GroupBy(t => t.Border)
            .ToDictionary(grp => grp.Key, grp => grp.Select(g => g.Tile).ToList());
        
        Corners = tiles.Where(tile => tile.Borders.Where(border => TileBorders[border].Count() == 1).Count() == 4);
        
        Grid[0, 0] = Corners.First();
        OrientCornerTile();
        LayHorizontalTiles(0);
        
        for (var i = 1; i < GridSize; i++)
        {
            LayVerticalTile(i);
            LayHorizontalTiles(i);
        }
    }
    
    private void OrientCornerTile()
    {
        var corner = Grid[0,0];
        var i = 0;
        while (!(TileBorders[corner.BorderTop].Count() == 1 && TileBorders[corner.BorderLeft].Count() == 1))
        {
            corner.Rotate();
            
            i++;
            if (i >= 4)
            {
                corner.Flip(0);
                i = 0;
            }
        }
    }
    
    private void LayHorizontalTiles(int row)
    {
        for (var i = 1; i < GridSize; i++)
        {
            var prev = Grid[row, i - 1];
            var current = TileBorders[prev.BorderRight].Where(t => t.Id != prev.Id).First();
            Grid[row, i] = current;

            var count = 0;
            while (prev.BorderRight != current.BorderLeft)
            {
                current.Rotate();

                count++;
                if (count >= 4)
                {
                    current.Flip(0);
                    count = 0;
                }
            }
        }
    }
    
    private void LayVerticalTile(int row)
    {
        var prev = Grid[row - 1, 0];
        var current = TileBorders[prev.BorderBottom].Where(t => t.Id != prev.Id).First();
        Grid[row, 0] = current;
        
        var count = 0;
        while (prev.BorderBottom != current.BorderTop)
        {
            current.Rotate();

            count++;
            if (count >= 4)
            {
                current.Flip(0);
                count = 0;
            }
        }
    }
    
    public void Rotate()
    {
        foreach (var tile in Tiles)
        {
            tile.Value.Rotate();
        }
        
        for (var i = 0; i < GridSize / 2; i++)
        {
            for (var j = i; j < GridSize - i - 1; j++)
            {
                (
                    Grid[               i, j               ],
                    Grid[               j, GridSize - i - 1],
                    Grid[GridSize - i - 1, GridSize - j - 1],
                    Grid[GridSize - j - 1, i               ]
                )
                =
                (
                    Grid[GridSize - j - 1, i               ],
                    Grid[               i, j               ],
                    Grid[               j, GridSize - i - 1],
                    Grid[GridSize - i - 1, GridSize - j - 1]
                );
            }
        }
    }
    
    public void Flip(int axis)
    {
        foreach (var tile in Tiles)
        {
            tile.Value.Flip(axis);
        }
        
        switch (axis)
        {
            case 0:
                for (var i = 0; i < GridSize; i++)
                {
                    for (var j = 0; j < GridSize / 2; j++)
                    {
                        (Grid[i, j], Grid[i, GridSize - j - 1]) = (Grid[i, GridSize - j - 1], Grid[i, j]);
                    }
                }
                break;
            case 1:
                for (var i = 0; i < GridSize / 2; i++)
                {
                    for (var j = 0; j < GridSize; j++)
                    {
                        (Grid[i, j], Grid[GridSize - i - 1, j]) = (Grid[GridSize - i - 1, j], Grid[i, j]);
                    }
                }
                break;
            default:
                throw new ArgumentException($"Invalid axis: {axis}.");
        }
    }
    
    public string ToString2()
    {
        var sb = new StringBuilder();
        for (var row = 0; row < Grid.GetLength(0); row++)
        {
            for (var pix = 0; pix < Grid[row,0].Pixels.Length; pix++)
            {
                for (var col = 0; col < Grid.GetLength(1); col++)
                {
                    sb.Append(new string(Grid[row, col].Pixels[pix]));
                    sb.Append(" ");
                }
                sb.AppendLine();
            }
            sb.AppendLine();
        }
        return sb.ToString();
    }
    
    public override string ToString()
    {
        var sb = new StringBuilder();
        for (var row = 0; row < Grid.GetLength(0); row++)
        {
            for (var prow = 1; prow < Grid[row,0].Pixels.Length - 1; prow++)
            {
                for (var col = 0; col < Grid.GetLength(1); col++)
                {
                    sb.Append(new string(Grid[row, col].Pixels[prow][1..^1]));
                }
                sb.AppendLine();
            }
        }
        return sb.ToString();
    }
}

In [5]:
var image = new Image(tiles);

In [6]:
image.Corners.Select(t => (long)t.Id)

index,value
0,1543
1,2887
2,2657
3,3989


In [7]:
image.Corners.Select(t => (long)t.Id).Aggregate((a, b) => a*b)

47213728755493

# --- Part Two ---

In [8]:
const string seaMonster = @"                  # 
#    ##    ##    ###
 #  #  #  #  #  #   ";

In [9]:
class SeaMonsterFinder
{    
    private string seaMonster;
    private int seaMonsterLength;
    private int seaMonsterHeight;
    private Image image;
    
    public int Count
    {
        get
        {
            var rotations = 0;
            while (CountSeaMonsters() == 0)
            {
                image.Rotate();

                rotations++;
                if (rotations >= 4)
                {
                    image.Flip(0);
                    rotations = 0;
                }
            }
            return CountSeaMonsters();
        }
    }
    
    public SeaMonsterFinder(string seaMonster, Image image)
    {
        this.seaMonster = seaMonster;
        this.image = image;
        
        seaMonsterLength = seaMonster.Split("\n").First().ToCharArray().Count();
        seaMonsterHeight = seaMonster.Split("\n").Count();
    }
    
    private int CountSeaMonsters()
    {
        var count = 0;
        var imageLength = image.ToString().Split("\n").First().ToCharArray().Count();
        var imageHeight = image.ToString().Split("\n").Count();
        
        for (var row = 0; row < imageHeight - seaMonsterHeight; row++)
        {
            for (var col = 0; col < imageLength - seaMonsterLength; col++)
            {
                if (HasSeaMonster(ImageSliceFrom(row, col)))
                {
                    count++;
                }
            }
        }
        
        return count;
    }
    
    private string ImageSliceFrom(int row, int col)
    {
        return string.Join("\n", image.ToString().Split("\n")
            .Select(line => line.ToCharArray())
            .Skip(row)
            .Take(seaMonsterHeight)
            .Select(line => new string(line.Skip(col).Take(seaMonsterLength).ToArray()))
        );
    }
    
    private bool HasSeaMonster(string input)
    {
        var seaMonsterArr = seaMonster.Split("\n").Select(line => line.ToCharArray()).ToArray();
        var inputArr = input.Split("\n").Select(line => line.ToCharArray()).ToArray();
        
        for (var i = 0; i < seaMonsterArr.Length; i++)
        {
            for (var j = 0; j < seaMonsterArr.First().Count(); j++)
            {
                if (seaMonsterArr[i][j] == ' ')
                {
                    continue;
                }
                else if (seaMonsterArr[i][j] != inputArr[i][j])
                {
                    return false;
                }
            }
        }
        
        return true;
    }
}

In [10]:
var seaMonsterFinder = new SeaMonsterFinder(seaMonster, image);
Regex.Matches(image.ToString(), "#").Count() - (Regex.Matches(seaMonster, "#").Count() * seaMonsterFinder.Count)

1599